In [31]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from copy import deepcopy
import pandas as pd

# 搜尋書名

取得書目 id


In [32]:
def set_up_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
    return webdriver.Chrome(service=Service("chromedriver.exe"), options=chrome_options)

In [33]:
def construct_serach_result_link(book_name: str) -> str:
    return f"https://bookshop.org/search?keywords={book_name}"

In [34]:
def get_soup_of_a_page(link: str) -> BeautifulSoup:
    driver = set_up_driver()
    driver.get(link)
    sleep(1)
    return BeautifulSoup(driver.page_source, "html.parser")

In [35]:
def get_book_id(book_name: str) -> str:
    soup_search_result = get_soup_of_a_page(construct_serach_result_link(book_name))
    book_id = soup_search_result.find("div", {"class": "search-result-card"}).find("a").get("href").split("/book/")[1]
    return book_id

# 進入書籍頁面


In [36]:
def construct_book_link(book_id: str) -> str:
    return f"https://bookshop.org/book/{book_id}"

In [37]:
def extract_book_info(soup_book: BeautifulSoup) -> dict:
    book_name_correct: str = soup_book.find("h1", itemprop="name").text  # 找到 <h1> 標籤且 itemprop 屬性為 'name'
    book_author = soup_book.find("div", {"class": "font-sans-medium text-gray text-sm"}).find("span", {"itemprop": "name"}).text.strip()
    book_info_of_the_option = {"Name": book_name_correct, "Author": book_author}
    book_info_of_the_option["Publisher"] = soup_book.find("div", itemprop="publisher").text.strip()
    book_info_of_the_option["Publish Date"] = soup_book.find("div", itemprop="datePublished").text.strip()
    book_info_of_the_option["Pages"] = int(soup_book.find("div", itemprop="numberOfPages").text.strip())
    book_info_of_the_option["Dimensions"] = soup_book.find("div", itemprop="size").text.strip()
    book_info_of_the_option["Language"] = soup_book.find("div", itemprop="inLanguage").text.strip()
    book_info_of_the_option["bookFormat"] = soup_book.find("div", itemprop="bookFormat").text.strip()
    book_info_of_the_option["ISBN"] = soup_book.find("div", itemprop="isbn").text.strip()
    book_info_of_the_option["Sale_Price"] = float(soup_book.find("meta", itemprop="price")["content"])
    book_info_of_the_option["Regular_Price"] = book_info_of_the_option["Sale_Price"]
    regular_price_tag = soup_book.find("span", class_="line-through text-primary")
    if regular_price_tag:
        book_info_of_the_option["Regular_Price"] = float(regular_price_tag.text.strip()[1:])
    return book_info_of_the_option

In [38]:
def extract_other_formats_links(soup_book_first_format: BeautifulSoup) -> dict:
    a_tags = soup_book_first_format.find_all("a", class_="flex flex-wrap rounded p-4 border border-border")
    other_formats_links = {}
    for a_tag in a_tags:
        href = a_tag.get("href")
        format_name = a_tag.find("div", class_="text-lg w-full truncate").text.strip()
        other_formats_links[format_name] = f"https://bookshop.org/{href}"
    return other_formats_links

In [39]:
def get_book_complete_information(target_name: str) -> dict:
    book_id = get_book_id(target_name)
    soup_book_first_format = get_soup_of_a_page(construct_book_link(book_id))
    book_info_of_first_format = extract_book_info(soup_book_first_format)
    other_formats_links = extract_other_formats_links(soup_book_first_format)
    book_complete_infomation = {book_info_of_first_format["bookFormat"]: deepcopy(book_info_of_first_format)}
    for format_name, link in other_formats_links.items():
        soup_format = get_soup_of_a_page(link)
        book_info_of_the_format = extract_book_info(soup_format)
        book_complete_infomation[format_name] = deepcopy(book_info_of_the_format)
    return book_complete_infomation

In [41]:
if __name__ == "__main__":
    target_book_name = "Caste: The Origins of Our Discontents"
    book_complete_infomation = get_book_complete_information(target_book_name)
    print(pd.DataFrame(book_complete_infomation))

{'Name': 'Caste: The Origins of Our Discontents', 'Author': 'Isabel Wilkerson', 'Publisher': 'Random House Trade', 'Publish Date': 'February 14, 2023', 'Pages': 544, 'Dimensions': '5.1 X 7.9 X 1.2 inches | 0.95 pounds', 'Language': 'English', 'bookFormat': 'Paperback', 'ISBN': '9780593230275', 'Sale_Price': 19.53, 'Regular_Price': 21.0}
                                           Paperback  \
Name           Caste: The Origins of Our Discontents   
Author                              Isabel Wilkerson   
Publisher                         Random House Trade   
Publish Date                       February 14, 2023   
Pages                                            544   
Dimensions      5.1 X 7.9 X 1.2 inches | 0.95 pounds   
Language                                     English   
bookFormat                                 Paperback   
ISBN                                   9780593230275   
Sale_Price                                     19.53   
Regular_Price                                